In [2]:
import numpy as np
import pandas as pd
import pickle
import time
import sys

pd.set_option("display.max_columns", 100)

In [2]:
1.0578 * 8582 + .68162 * 5026

12503.86172

In [39]:
### Use this to view the accounts_template.pickle file

# raw price data has been munged and is in a picKle file ready for use.
filename = "accounts_template.pickle"

# open the existing raw data from the pickle file
infile = open(filename, "rb")
df = pickle.load(infile)
infile.close()
# df

Munge historical data, create prices dataframe.

In [41]:
# raw data has been munged and is in a picle file ready for use. This list
# out full information including
filename = "historical_prices.pickle"

# open the existing raw data from the pickle file
infile = open(filename, "rb")
hist = pickle.load(infile)
infile.close()

# the securities start at different times. the latest start is
# 2001-09-06. drop all securities before then.
# start analysis at the beginning of 2002
# end at '2018-12-31'
hist = hist[hist["trade_date"] >= "2002-01-01"]
hist = hist[hist["trade_date"] <= "2018-12-31"]

# reset the index inplace
hist.reset_index(drop=True, inplace=True)

# eliminate securities not used
hist = hist[hist["security"].isin(["XBB", "XIC", "TD_Bond", "TD_CDN_Equity"])]

# change nan values to 0
hist["dividends"].fillna(0, inplace=True)

initialize

In [42]:
# transform the hist df. it needs to be reshaped
# create a dictionary of dataframes and then concat them on axis 1
list_of_dfs = {}
securities = ["XBB", "XIC", "TD_Bond", "TD_CDN_Equity"]

# create four dateframes form hist, drop the security columns, and then concat
for sec, df in zip(securities, ["xbb_df", "xic_df", "td_bond_df", "td_eq_df"]):
    list_of_dfs[df] = (
        hist[hist.loc[:, "security"] == sec]
        .set_index("trade_date")
        .drop(["security"], axis=1)
    )
df = pd.concat(list_of_dfs.values(), axis=1)

In [43]:
# make a list of the headers of the columns from hist
# drop the security and trade date from the header list
headers = hist.columns.values
head = list(headers)

head.remove("security")
head.remove("trade_date")

In [44]:
# create the new column names and add to df.
li = []
for s in securities:
    for h in head:
        li.append(s + "-" + h)
df.columns = li

In [45]:
# initialize all of the columns

# insert trading columns for mutual funds with 0 value
df.insert(1, "TD_Bond-unit_traded", 0)
df.insert(2, "TD_Bond-unit", 0)
df.insert(3, "TD_Bond-value", 0)
df.insert(4, "TD_CDN_Equity-unit_traded", 0)
df.insert(5, "TD_CDN_Equity-unit", 0)
df.insert(6, "TD_CDN_Equity-value", 0)

# These are the unitS traded
df.insert(0, "XIC-unit_traded", 0)
df.insert(0, "XBB-unit_traded", 0)

# for the first transaction, units = units_traded
df.insert(1, "XBB-unit", 0)
df.insert(2, "XBB-value", 0)
df.insert(4, "XIC-unit", 0)
df.insert(5, "XIC-value", 0)

# acb columns 
df.insert(1, "XBB-acb", 0)
df.insert(2, "XIC-acb", 0)
df.insert(3, "TD_Bond-acb", 0)
df.insert(4, "TD_CDN_Equity-acb", 0)


# iniitial set up of the dataframe with no rebalancing yet. Just an initial
# investment growing with the market. All dividends to cash. No Mutual Funds
df.insert(0, "cash", 0)
df.insert(1, "dep_with", 0)
df.insert(2, "market_value", 0)
df.insert(3, "total_value", 0)
df.insert(4, "total_value_norm", 0)
df.insert(5, "tax_total", 0)
df.insert(6, "value_after_tax", 0)
df.insert(7, "value_after_tax_norm", 0)
df.insert(8, "costs", 0)
df.insert(9, "rebalanced", False)
df.insert(10, "cash_allocation", 0)
df.insert(11, "fixed_income_allocation", 0)
df.insert(12, "equity_allocation", 0)
df.insert(13, "cash_total", 0)
df.insert(14, "fixed_income_total", 0)
df.insert(15, "equity_total", 0)
df.insert(16, "tax_accrued", 0)
df.insert(17, "tax_dividend", 0)
df.insert(18, "tax_gain", 0)
df.insert(19, "tax_realized", 0)





# dividends to cash
df.insert(2, "dividends", 0)

# track cash flow for purchases and sales
df.insert(3, "purchases", 0)
df.insert(4, "sales", 0)

In [46]:
# the dataframe template is built, sort the columns.
columnsTitles = [
    "cash",
    "dep_with",
    "dividends",
    "purchases",
    "sales",
    "market_value",
    "total_value",
    "total_value_norm",
    "tax_accrued",
    "tax_dividend",
    "tax_gain",
    "tax_realized",
    "tax_total",
    "value_after_tax",
    "value_after_tax_norm",
    "costs",
    "rebalanced",
    "cash_allocation",
    "fixed_income_allocation",
    "equity_allocation",
    "cash_total", 
    "fixed_income_total", 
    "equity_total",
    "XBB-nav_per_share",
    "XBB-unit_traded",
    "XBB-unit",
    "XBB-value",
    "XBB-dividends",
    "XBB-acb",
    "XBB-asset_class",
    "XBB-fund_type",
    "XIC-nav_per_share",
    "XIC-unit_traded",
    "XIC-unit",
    "XIC-value",
    "XIC-dividends",
    "XIC-acb",
    "XIC-asset_class",
    "XIC-fund_type",
    "TD_Bond-nav_per_share",
    "TD_Bond-unit_traded",
    "TD_Bond-unit",
    "TD_Bond-value",
    "TD_Bond-dividends",
    "TD_Bond-acb",
    "TD_Bond-asset_class",
    "TD_Bond-fund_type",
    "TD_CDN_Equity-nav_per_share",
    "TD_CDN_Equity-unit_traded",
    "TD_CDN_Equity-unit",
    "TD_CDN_Equity-value",
    "TD_CDN_Equity-dividends",
    "TD_CDN_Equity-acb",
    "TD_CDN_Equity-asset_class",
    "TD_CDN_Equity-fund_type",
]

df = df.reindex(columns=columnsTitles)

In [47]:
# pickle the dataframe for use in other analysis
with open("accounts_template.pickle", "wb") as f:
    pickle.dump(df, f)